In [33]:
import os
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
import numpy as np
from PIL import Image
import easyocr
import json
import re

In [82]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [17]:
path = "./input"

for dir,_, files in os.walk(path):
    for pdfFile in files:
        pdf_path = f'{path}/{pdfFile}'
        with pdfplumber.open(pdf_path) as pdf:
            # print(pdf.pages)
            for x in pdf.pages:
                print(x.extract_text(), end='\n\n')

Overview
Foundation Level Extensions
Version 1.0
International Software Testing Qualifications Board
Copyright Notice
This document may be copied in its entirety, or extracts made, if the source is acknowledged.

International
Overview
Software Testing
Foundation Level Extension – Agile Tester
Qualifications Board
Copyright © International Software Testing Qualifications Board (hereinafter called ISTQB®).
Version 2014 Page 2 of 12 May 31, 2014
© International Software Testing Qualifications Board

International
Overview
Software Testing
Foundation Level Extension – Agile Tester
Qualifications Board
Revision History
Version Date Remarks
0.1 18 JUNE 2013 Initial version
0.2 23 JULY 2013 WG reviewed and confirmed
0.3 6 NOV 2013 amended population and diagram
0.7 11 DEC 2013 Amended Business Outcomes and Chapters matching
0.8 20 DEC 2013 Working group updates on 0.7
1.0 31 MAY 2014 GA release for Agile Extension
Version 2014 Page 3 of 12 May 31, 2014
© International Software Testing Qualif

In [91]:
pdf5 = f'input/file03.pdf'
with pdfplumber.open(pdf5) as pdf:
    # print(pdf.pages)
    for x in pdf.pages:
        print(x.extract_text())
        # for line in x.extract_text():
        #     print(line)
            # print(line['chars'])

Ontario’s Libraries
Working Together
RRRRFFFFPPPP:::: RRRReeeeqqqquuuueeeesssstttt ffffoooorrrr PPPPrrrrooooppppoooossssaaaallll
To Present a Proposal for Developing
the Business Plan for the Ontario
Digital Library
March 21, 2003
The Ontario Digital Library will make Ontario a better place to study, work and live by ensuring that
all Ontario citizens have access to the knowledge and learning supports needed to be life-long
learners and effective contributors towards Ontario’s prosperity.
Ontario’s Digital Library
A Critical Component for Implementing Ontario’s Road Map to
Prosperity Strategy
Summary
The purpose of this Request for Proposal (RFP) is to invite firms and/or consultants to
present a proposal for developing the business plan for the Ontario Digital Library (ODL).
The ODL will deliver high-quality library electronic content to all Ontario residents in order to
assist people as they learn, work, and enhance their quality of life. The business plan to be
developed is to docum

In [92]:
pages = convert_from_path(pdf5, poppler_path="C:/Program Files/poppler/poppler-24.08.0/Library/bin")
for page in pages:
    text = pytesseract.image_to_string(page)
    print(text)

Ontario’s Libraries
Working Together

RFP: Request for Proposal

To Present a Proposal for Developing
the Business Plan for the Ontario
Digital Library

March 21, 2003

Connecting
Ontarians!

The
Ontario
Digital Library

The Ontario Digital Library will make Ontario a better place to study, work and live by ensuring that
all Ontario citizens have access to the knowledge and learning supports needed to be life-long
learners and effective contributors towards Ontario's prosperity.


Ontario’s Digital Library

A Critical Component for Implementing Ontario’s Road Map to
Prosperity Strategy

Summary

The purpose of this Request for Proposal (RFP) is to invite firms and/or consultants to
present a proposal for developing the business plan for the Ontario Digital Library (ODL).
The ODL will deliver high-quality library electronic content to all Ontario residents in order to
assist people as they learn, work, and enhance their quality of life. The business plan to be
developed is to document a

In [96]:
# pdf5 = 'input/file05.pdf'
# pages = convert_from_path(pdf5, poppler_path="C:/Program Files/poppler/poppler-24.08.0/Library/bin")
# reader = easyocr.Reader(['en'])

# for i, page_img in enumerate(pages):
#     page_img = np.array(page_img)
#     result = reader.readtext(page_img)
#     for item in result:
#         print(item[1])

In [97]:
def extract_pdf_lines(pdf_path):
    structured_blocks = []
    page_sizes = {}
    order = 0

    with pdfplumber.open(pdf_path) as pdf:
        # Page sizes for all pages
        for page_idx, page in enumerate(pdf.pages, 1):
            page_sizes[str(page_idx)] = {
                "width": page.width,
                "height": page.height
            }

        # Extract lines per page
        for page_idx, page in enumerate(pdf.pages, 1):
            # This function returns a list of dicts, one per line
            for line in page.extract_text_lines():
                # print(line['text'])
                block = {
                    "id": order,
                    "text": line['text'],
                    "box": [line["x0"], line["top"], line["x1"], line["bottom"]],
                    "fontname": line.get("fontname", ""),
                    "font_size": line.get("size", 0),
                    "page": page_idx,
                    "order": order
                }
                structured_blocks.append(block)
                order += 1

    result = {
        "pages": page_sizes,
        "contents": structured_blocks
    }
    return result

In [100]:
def extract_pdf_paragraphs(pdf_path):
    structured_blocks = []
    page_sizes = {}
    order = 0

    with pdfplumber.open(pdf_path) as pdf:
        # Save sizes
        for page_idx, page in enumerate(pdf.pages, 1):
            page_sizes[str(page_idx)] = {
                "width": page.width,
                "height": page.height
            }

        for page_idx, page in enumerate(pdf.pages, 1):
            lines = list(page.extract_text_lines())
            
            if not lines:
                continue

            # Sort lines top-to-bottom just in case
            lines = sorted(lines, key=lambda l: l['top'])

            current_paragraph = ""
            current_box = [lines[0]["x0"], lines[0]["top"], lines[0]["x1"], lines[0]["bottom"]]

            # Calculate median line spacing for the page
            if len(lines) > 1:
                line_gaps = [lines[i+1]['top'] - lines[i]['bottom'] for i in range(len(lines)-1)]
                median_gap = sorted(line_gaps)[len(line_gaps)//2]
            else:
                median_gap = 5

            # print(median_gap)

            for i, line in enumerate(lines):
                text = line["text"]
                # For the first line (already started)
                if i == 0:
                    current_paragraph = text
                    continue

                prev_line = lines[i-1]
                gap = line["top"] - prev_line["bottom"]

                # Heuristic: large gap? Start new paragraph
                if gap > 1.5 * median_gap: # Try tweaking 1.5 up or down!
                    block = {
                        "box": current_box,
                        "text": current_paragraph.strip(),
                        "page": page_idx,
                        "id": order,
                        "fontname": line['chars'][0]['fontname'],
                        "order": order
                    }
                    structured_blocks.append(block)
                    order += 1
                    
                    current_paragraph = text
                    current_box = [line["x0"], line["top"], line["x1"], line["bottom"]]
                    current_fontname = line.get("fontname", "")
                else:
                    # Continue current paragraph
                    current_paragraph += " " + text
                    # Expand bounding box to include new line
                    current_box[0] = min(current_box[0], line["x0"])
                    current_box[1] = min(current_box[1], line["top"])
                    current_box[2] = max(current_box[2], line["x1"])
                    current_box[3] = max(current_box[3], line["bottom"])

            # After loop: save last paragraph
            if current_paragraph.strip():
                block = {
                    "box": current_box,
                    "text": current_paragraph.strip(),
                    "page": page_idx,
                    "id": order,
                    "fontname": current_fontname,
                    "order": order
                }
                structured_blocks.append(block)
                order += 1

    result = {
        "pages": page_sizes,
        "contents": structured_blocks
    }
    return result

In [102]:
input_folder = "./input"
output_folder = "./output"

for fname in os.listdir(input_folder):
    if fname.lower().endswith(".pdf"):
        pdf_path = os.path.join(input_folder, fname)
        out_path = os.path.join(output_folder, os.path.splitext(fname)[0] + ".json")
        structured = extract_pdf_paragraphs(pdf_path)
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(structured, f, indent=2)
        print(f"Extracted structure for {fname} -> {out_path}")

Extracted structure for file01.pdf -> ./output\file01.json
Extracted structure for file02.pdf -> ./output\file02.json
Extracted structure for file03.pdf -> ./output\file03.json
Extracted structure for file04.pdf -> ./output\file04.json
Extracted structure for file05.pdf -> ./output\file05.json


<pre>
Model input: 
{
    "box": [_, _, _, _],
    "text": "______",
    "page": _,
    "id": _
}

Model Output:
{
    "label": "_____",
    "linking": [[_, _]],
    "order": 
}
</pre>

In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
import xgboost as xgb
import joblib

# Folders
input_json_dir = './output'
ground_truth_dir = './ground_truth'

# Data prep
texts = []
features_list = []
labels = []

file_names = [f for f in os.listdir(input_json_dir) if f.endswith('.json')]
for file_name in file_names:
    with open(os.path.join(input_json_dir, file_name), 'r', encoding='utf-8') as f:
        input_blocks = json.load(f).get('contents', [])
    with open(os.path.join(ground_truth_dir, file_name), 'r', encoding='utf-8') as f:
        gt_blocks = json.load(f).get('contents', [])
    gt_map = {blk['id']: blk['label'] for blk in gt_blocks if 'id' in blk and 'label' in blk}
    for blk in input_blocks:
        texts.append(blk.get('text', ''))
        features_list.append(blk.get('box', [0, 0, 0, 0]) + [blk.get('page', 0)])
        labels.append(gt_map.get(blk['id'], 'O')) # 'O' for unknown

# Encode features/labels
tfidf = TfidfVectorizer(max_features=2000)
X_text = tfidf.fit_transform(texts)

X_num = np.array(features_list)
X = hstack([X_text, X_num])
lab_enc = LabelEncoder()
y = lab_enc.fit_transform(labels)

# Train model
dtrain = xgb.DMatrix(X, label=y)
params = {'objective': 'multi:softmax', 'num_class': len(lab_enc.classes_), 'max_depth': 6, 'eta': 0.3}
bst = xgb.train(params, dtrain, num_boost_round=1000)

# Save model and encoders
bst.save_model('xgb_heading_label.model')
joblib.dump(lab_enc, 'label_encoder.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

C:\Users\soham\AppData\Local\Temp\ipykernel_4660\1340312921.py:43: UserWarning: [03:37:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  bst.save_model('xgb_heading_label.model')


['tfidf_vectorizer.pkl']

In [118]:
test_pdf_dir = './input'
test_output_dir = './test_output'
os.makedirs(test_output_dir, exist_ok=True)

for fname in os.listdir(test_pdf_dir):
    if fname.lower().endswith('.pdf'):
        in_pdf = os.path.join(test_pdf_dir, fname)
        out_json = os.path.join(test_output_dir, os.path.splitext(fname)[0] + ".json")
        # Use the function you prefer:
        doc = extract_pdf_paragraphs(in_pdf)
        with open(out_json, "w", encoding="utf-8") as f:
            json.dump(doc, f, indent=2)
        print(f"Processed: {fname} -> {out_json}")

Processed: file01.pdf -> ./test_output\file01.json
Processed: file02.pdf -> ./test_output\file02.json
Processed: file03.pdf -> ./test_output\file03.json
Processed: file04.pdf -> ./test_output\file04.json
Processed: file05.pdf -> ./test_output\file05.json


In [119]:
bst = xgb.Booster()
bst.load_model('xgb_heading_label.model')
tfidf = joblib.load('tfidf_vectorizer.pkl')
lab_enc = joblib.load('label_encoder.pkl')

for fname in os.listdir(test_output_dir):
    if fname.endswith('.json'):
        with open(os.path.join(test_output_dir, fname), "r", encoding="utf-8") as f:
            doc = json.load(f)
        blocks = doc["contents"]
        texts = [b["text"] for b in blocks]
        X_text = tfidf.transform(texts)
        X_num = np.array([b["box"] + [b["page"]] for b in blocks])
        X = hstack([X_text, X_num])
        dtest = xgb.DMatrix(X)
        y_pred = bst.predict(dtest).astype(int)
        pred_labels = lab_enc.inverse_transform(y_pred)
        for block, pred in zip(blocks, pred_labels):
            block["predicted_label"] = pred
        with open(os.path.join(test_output_dir, fname.replace(".json", "_pred.json")), "w", encoding="utf-8") as fout:
            json.dump(doc, fout, indent=2)
        print(f"Predicted labels for {fname}")

Predicted labels for file01.json
Predicted labels for file02.json
Predicted labels for file03.json
Predicted labels for file04.json
Predicted labels for file05.json
